# Exploratory Analysis of Amazon-Google Entity Product Matching Data Set

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
# Navigate to current EDA directory as the relative path point
%pwd

'c:\\Users\\seans\\Documents\\neural_entity_matching\\eda'

In [8]:
amz_train = pd.read_csv("../data/amazon_google/Amazon_train.csv",index_col="id")
g_train = pd.read_csv("../data/amazon_google/Amazon_train.csv",index_col="id")

matches_train = pd.read_csv("../data/amazon_google/AG_perfect_matching_train.csv",index_col = ["idAmazon","idGoogleBase"])
matches_test = pd.read_csv("../data/amazon_google/AG_perfect_matching_test.csv",index_col = ["idAmazon","idGoogleBase"])

SyntaxError: invalid syntax (<ipython-input-8-9a2fa1444115>, line 1)

In [3]:
amz_train.describe()

,Unnamed: 0,price
count,1113.00000,1113.000000
mean,556.00000,385.618985
std,321.43973,3423.399301
min,0.00000,0.000000
25%,278.00000,19.950000
50%,556.00000,39.950000
75%,834.00000,145.390000
max,1112.00000,101515.550000
